In [4]:
import pandas as pd
import numpy as np
#import matplotlib as plt
import requests
import pdfplumber
import re
#import nltk
import statistics

In [5]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

In [6]:
def get_prices(pdf,page, dictionary):
    doc = pdfplumber.open(pdf)
    prod_df=pd.DataFrame(doc.pages[page].extract_table())
    
    def find_prices(df,word):
        for x in range(df.shape[0]):
            for y in range(df.shape[1]):
                if word in str(df[y][x]):
                    return y
    
    def find_index(df,dictionary):
        index_col=list()
        for x in range(df.shape[0]):
            for y in range(df.shape[1]):
                if df[y][x]=='None':
                    pass
                else:
                    for key in dictionary.keys():
                        if df[y][x] in dictionary[key]:
                            index_col.append(y)
        return int(statistics.median(index_col))
    
    def clean_df(df):
        for x in range(df.shape[0]):
            for y in df.columns:
                if str(df[y][x])=='' or str(df[y][x])=='None':
                    df[y][x]=np.nan
        return df.dropna(axis='columns', how='all').dropna(axis='rows', how='all')

    
    prices_col=find_prices(prod_df, 'Euros')
    index_col=find_index(prod_df, house_credit_com)
    
    prices_df=pd.DataFrame()
    prices_df['Commissions']=prod_df[index_col]
    prices_df['Prices']=prod_df[prices_col]
    
    return clean_df(prices_df)

In [7]:
get_prices('data/bic.pdf', 8, house_credit_com)

,Commissions,Prices
2,2.1. Crédito à habitação e outros créditos hip...,NaN
5,NaN,Euros \n(Mín/Máx)
6,Crédito à Habitação com Garantia Hipotecária N...,NaN
7,Comissões iniciais,NaN
8,1. Comissão de processo,NaN
9,Habitação Própria,"260,00"
10,Habitação Própria (oferta sénior),"260,00"
11,Troca de Casa (Fora de comercialização),"260,00"
12,Troca de Banco,"260,00"
13,Habitação para Arrendamento,"260,00"
